In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
training_summary.r2

0.027839179518600154

In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

In [13]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [14]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [15]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                347|
|   mean|0.42068210636959574|
| stddev| 10.369587269240675|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [16]:
test_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 154|
|   mean|-0.11217786357505664|
| stddev|  10.224369828757306|
|    min| -26.805483428483072|
|    max|  24.290551295953957|
+-------+--------------------+



In [17]:
correct_model = lr.fit(train_data)

In [18]:
test_results = correct_model.evaluate(test_data)

In [19]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -29.06358657132393|
|  -27.8583300537824|
|-22.278543667174752|
|-20.777559329256068|
|-21.029343533841967|
|-16.815896768638623|
|-17.694374177278426|
| -20.34322758836456|
| -18.15151810814432|
| -16.03026093791407|
| -18.51500855821421|
| -15.12040768044456|
|-12.583339537345271|
|-15.344596173205652|
|-17.351754760927395|
|-12.225079416588796|
|-16.278893922411175|
|-14.546831073563895|
|-11.837790734400954|
|-10.862830331983789|
+-------------------+
only showing top 20 rows



In [20]:
test_results.rootMeanSquaredError

10.299501439837265

In [21]:
unlabeled_data = test_data.select('features')

In [22]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [23]:
predictions = correct_model.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|   2.258103142840858|
|(10,[0,1,2,3,4,5,...|   4.908504117586325|
|(10,[0,1,2,3,4,5,...|  0.8461559030089449|
|(10,[0,1,2,3,4,5,...|  0.5654820702973963|
|(10,[0,1,2,3,4,5,...|   1.362024918470247|
|(10,[0,1,2,3,4,5,...|  -2.352395854325137|
|(10,[0,1,2,3,4,5,...|-0.10925201138608892|
|(10,[0,1,2,3,4,5,...|  3.2778279624885447|
|(10,[0,1,2,3,4,5,...|  1.1250258439347705|
|(10,[0,1,2,3,4,5,...| -0.6888358956910225|
|(10,[0,1,2,3,4,5,...|   1.822801536903105|
|(10,[0,1,2,3,4,5,...| -0.9652513605769308|
|(10,[0,1,2,3,4,5,...|  -2.765531618033983|
|(10,[0,1,2,3,4,5,...|  0.5818379202745245|
|(10,[0,1,2,3,4,5,...|   3.022776251851952|
|(10,[0,1,2,3,4,5,...| -1.6420084785699711|
|(10,[0,1,2,3,4,5,...|   2.506452360708305|
|(10,[0,1,2,3,4,5,...|  1.5069030094592801|
|(10,[0,1,2,3,4,5,...|  -1.140057990991149|
|(10,[0,1,2,3,4,5,...| -1.910396